Dataset

Training 
  Positive : 26974, 
  Negative : 27470

Validation 
  Positive : 2150, 
  Negative : 2136

In [ ]:
import json
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# from tensorflow import feature_column
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/2/training'

data-edit    training.txt	 Train_negative.txt   Train_positive.txt
train1.csv   training.xlsx	 Train_negative.xlsx
train1.xlsx  Train_negative.csv  Train_positive.csv


In [ ]:
#utitliy functions 

import os
import sys
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import datetime

output_path="/content/drive/MyDrive/Colab Notebooks/2/results/TargetNet_training/vgg_novel"

def set_output(string):
    """ set output configurations """
    output, save_prefix = sys.stdout, None
    if output_path is not None:
        save_prefix = output_path
        if not os.path.exists(save_prefix):
            os.makedirs(save_prefix, exist_ok=True)
        output = open( output_path + string + ".txt", "a")

    return output, save_prefix

def Print(string, output, newline=False, timestamp=True):
    """ print to stdout and a file (if given) """
    if timestamp:
        time = datetime.datetime.now()
        line = '\t'.join([str(time.strftime('%m-%d %H:%M:%S')), string])
    else:
        time = None
        line = string

    print(line, file=sys.stderr)
    if newline: print("", file=sys.stderr)

    if not output == sys.stdout:
        print(line, file=output)
        if newline: print("", file=output)

    output.flush()
    return time

In [ ]:
rol = []
for x in range(48):
  rol.append([0,0,0])
print(rol)
print(type(rol))



[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
<class 'list'>


In [ ]:
def encode_RNA(mirna_seq, mirna_esa, cts_rev_seq, cts_rev_esa,lenMirna):
    """ one-hot encoder for RNA sequences with/without extended seed alignments """

    mRnaseq = cts_rev_seq
    miRnaseq = mirna_seq
    mRnaesa = cts_rev_esa
    miRnaesa = mirna_esa

    seq_data = []
    seqrol = []

    for i in range(len(mRnaseq)):
        if i<5 or i>(len(miRnaseq)-1+5):
            rolTemp=[]
            for x in range(48):

              if x in range(4):
                if mRnaseq[i] == 'A':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(4,8):
                if mRnaseq[i] == 'U':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(8,12):
                if mRnaseq[i] == 'C':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(12,16):
                if mRnaseq[i] == 'G':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(16,20):
                if mRnaseq[i] == '-':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              else:
                rolTemp.append([0,0,0])

            # print("area1", np.array(rolTemp).shape)
            seq_data.append(rolTemp)
        else:
            if i<15:
                if mRnaesa[i-5] == 'A':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])


                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
                    # print("area2", np.array(rolTemp).shape)

                if mRnaesa[i-5] == 'U':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4,8):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])

                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area3", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)
          
                if mRnaesa[i-5] == 'C':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(8,12):
                          rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                          

                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
                    

                if mRnaesa[i-5] == 'G':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(12,16):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaesa[i-5] == '-':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(16,20):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area6", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)
          
            else:
                if mRnaseq[i] == 'A':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == 'U':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4,8):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
                
                if mRnaseq[i] == 'C':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(8,12):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == 'G':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(12,16):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == '-':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(16,20):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
    
    for p in range(8):
      seq_data.append(rol)
    x= np.array(seq_data)
    
    return x

In [ ]:
!pip install bio
import sys
import numpy as np
from Bio import pairwise2

# import torch

#Loading the datasets
Positive_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/partial/miRAW_positive.csv'
Negative_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/partial/miRAW_negative.csv'
TestingData = '/content/drive/MyDrive/Colab Notebooks/2/data/txt/novel/9/9_part2.txt'


class miRNA_CTS_dataset():
    """ Pytorch dataloader for miRNA-CTS pair data """
    def __init__(self, X, labels, set_idxs, set_labels):
        self.X = X
        self.labels = labels
        self.set_idxs = set_idxs
        self.set_labels = set_labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.X[i], self.labels[i], self.set_idxs[i]

        

def reverse(seq):
    """ reverse the given sequence """
    seq_r = ""
    for i in range(len(seq)):
        seq_r += seq[len(seq) - 1 - i]
    return seq_r


score_matrix = {}  # Allow watson-crick & wobble
for c1 in 'ACGU':
    for c2 in 'ACGU':
        if (c1, c2) in [('A', 'U'), ('U', 'A'), ('G', 'C'), ('C', 'G')]:
            score_matrix[(c1, c2)] = 1
        elif (c1, c2) in [('U', 'G'), ('G', 'U')]:
            score_matrix[(c1, c2)] = 1
        else:
            score_matrix[(c1, c2)] = 0


def extended_seed_alignment(mi_seq, cts_r_seq):
    """ extended seed alignment """
    alignment = pairwise2.align.globaldx(mi_seq[:10], cts_r_seq[5:15], score_matrix, one_alignment_only=True)[0]
    mi_esa = alignment[0]
    cts_r_esa = alignment[1]
    esa_score = alignment[2]
    return mi_esa, cts_r_esa, esa_score


def get_dataset_from_configs(data_cfg,split_idx=None):
  
    FILE = open(data_cfg, "r")
    lines = FILE.readlines()
    FILE.close()

    X, labels, set_idxs, set_labels = [], [], [], []
    set_idx = 0
    for l, line in enumerate(lines[1:]):
      if split_idx=="test":
        tokens = line.strip().split(",")
      else:
        tokens = line.strip().split("\t")
      mirna_id, mirna_seq, mrna_id, mrna_seq = tokens[:4]
      label = float(tokens[4]) if len(tokens) > 4 else 0
      
      mirna_seq = mirna_seq.upper().replace("T", "U")
      mrna_seq = mrna_seq.upper().replace("T", "U")
      mrna_rev_seq = reverse(mrna_seq)

      for pos in range(len(mrna_rev_seq) - 40 + 1):
          mirna_esa, cts_rev_esa, esa_score = extended_seed_alignment(mirna_seq, mrna_rev_seq[pos:pos+40])
          if esa_score < 6: continue
          X.append((encode_RNA(mirna_seq, mirna_esa,
                                                mrna_rev_seq[pos:pos+40], cts_rev_esa,len(mrna_seq))))
          labels.append(tf.expand_dims(np.array(label), 0))
          set_idxs.append(tf.expand_dims(np.array(set_idx), 0))

      set_labels.append(label)
      set_idx += 1
      if set_idx % 5 == 0:
          print('# {} {:.1%}'.format(split_idx, l / len(lines[1:])), end='\r', file=sys.stderr)
    print(' ' * 150, end='\r', file=sys.stderr)

    dataset = miRNA_CTS_dataset(X, labels, set_idxs, np.array(set_labels))
      
    return X, labels, set_idxs, set_labels

output, save_prefix = set_output("train_model_log")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 272 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 81.6 MB/s 


/usr/local/lib/python3.7/dist-packages/Bio/pairwise2.py:283: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  BiopythonDeprecationWarning,


In [ ]:
import random 

def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y;

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out

In [ ]:
start = Print(" ".join(['start loading datasets']), output)

Positive_X, Positive_y, Positive_set_idxs, Positive_set_labels = get_dataset_from_configs(Positive_Data,split_idx="train")
Negative_X, Negative_y, Negative_set_idxs, Negative_set_labels = get_dataset_from_configs(Negative_Data, split_idx="val")

test_X, test_y, test_set_idxs, test_set_labels = get_dataset_from_configs(TestingData)

test_X = np.array(test_X)
test_y = np.array(test_y)
test_set_idxs = np.array(test_set_idxs)

folds = 10
Positive_X_Slices = chunkIt(Positive_X, folds);
Positive_y_Slices = chunkIt(Positive_y, folds);

Negative_X_Slices = chunkIt(Negative_X, folds);
Negative_y_Slices = chunkIt(Negative_y, folds);

print(len(Positive_X_Slices), 'train examples')
print(len(Positive_y_Slices), 'validation examples')
print(len(Negative_X_Slices), 'test examples')


11-22 04:24:54	start loading datasets


10 train examples
10 validation examples
10 test examples


In [ ]:
print(np.array(Positive_X).shape)

(28064, 48, 48, 3)


In [ ]:
from tensorflow.keras.optimizers import SGD,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, LSTM, ConvLSTM2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input,Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation, LSTM, ConvLSTM2D, Lambda, Reshape, BatchNormalization, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,TensorBoard,TerminateOnNaN, LearningRateScheduler
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.layers import Lambda, Reshape, DepthwiseConv2D, ZeroPadding2D, Add, MaxPooling2D,Activation, Flatten, Conv2D, Dense, Input, Dropout, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

def getModel():
    
    # Input layer
    baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(48, 48, 3)))
    for layer in baseModel.layers:
        layer.trainable = False
    x = baseModel.output
    
    print(baseModel.output)
    # LSTM layer
    x = Reshape((1, 512))(x)
    x = ((LSTM(512, activation="relu", return_sequences=True, trainable=False)))(x)
    x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
    
    # FC layer
    x = Flatten(name="flatten")(x)
    
    # fc1 layer
    x = Dense(units=4096, activation='relu')(x)
    x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)

    # fc2 layer
    x = Dense(units=4096, activation='relu')(x)
    x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
    
    # Output layer
    output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=baseModel.input, outputs=output)
    opt = RMSprop(learning_rate=0.02, rho=0.9, epsilon=1e-06, clipnorm=1.)
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
    
    return model

In [ ]:
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc
#from tensorflow.compat.v1 import Session
from tensorflow.keras import losses

tf.compat.v1.disable_eager_execution()

def calculateScore(X, y, model):
    
    score = model.evaluate(X,y)
    pred_y = model.predict(X)

    accuracy = score[1];

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0;
        else:
            tempLabel[i] = 1;
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

    sensitivity = recall_score(y, tempLabel)
    specificity = TN / float(TN+FP)
    MCC = matthews_corrcoef(y, tempLabel)

    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)

    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None;

    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)

    with tf.compat.v1.Session():
        lossValue = losses.binary_crossentropy(y_true, y_pred).eval()

    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea, 'precision' : precision, 'F1' : F1Score, 'fpr' : fpr, 'tpr' : tpr, 'thresholds' : thresholds, 'lossValue' : lossValue}
    # return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea, 'precision' : precision, 'F1' : F1Score, 'lossValue' : lossValue}



In [ ]:
from scipy import interp
import matplotlib.pyplot as plt

def analyze(temp, OutputDir):

    # temp = None
    # with open(dataFile, 'rb') as file:
    #        temp = pickle.load(file)

    trainning_result, validation_result, testing_result = temp;

    file = open(OutputDir + '/performance.txt', 'w')

    index = 0
    for x in [trainning_result, validation_result, testing_result]:


        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        index += 1;

        file.write(title +  'results\n')


        for j in ['sn', 'sp', 'acc', 'MCC', 'AUC', 'precision', 'F1', 'lossValue']:

            total = []

            for val in x:
                total.append(val[j])

            file.write(j + ' : mean : ' + str(np.mean(total)) + ' std : ' + str(np.std(total))  + '\n')

        file.write('\n\n______________________________\n')
    file.close();

    index = 0

    for x in [trainning_result, validation_result, testing_result]:

        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)

        i = 0

        for val in x:
            tpr = val['tpr']
            fpr = val['fpr']
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

            i += 1

        print;

        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        plt.savefig( OutputDir + '/' + title +'ROC.png')
        plt.close('all');

        index += 1;

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.keras import regularizers, Input, Model, Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, Flatten, Dropout, Dense, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tf.compat.v1.disable_eager_execution()

OutputDir = '/content/drive/MyDrive/Colab Notebooks/2/finals/vgg_lstm/vgg16_17_lstm/results'

trainning_result = []
validation_result = []
testing_result = []

epochs = 50
# batches = [512, 256, 200, 100]
learning_rate = [0.01]
dropout = [0.2]
batches = [256]
# lr = 0.02
# dout = 0.2
fils = 320
ksize = 12
acc = 0


for test_index in range(folds):
  valid_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
  valid_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))

  validation_index = (test_index+1) % folds;

  valid_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
  valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

  start = 0;

  for val in range(0, folds):
    if val != test_index and val != validation_index:
      start = val;
      break;

  train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
  train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

  for i in range(0, folds):
    if i != test_index and i != validation_index and i != start:
      tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
      tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

      train_X = np.concatenate((train_X, tempX))
      train_y = np.concatenate((train_y, tempy))

  test_X, test_y = shuffleData(test_X,test_y);
  valid_X,valid_y = shuffleData(valid_X,valid_y);
  train_X,train_y = shuffleData(train_X,train_y);

  model = getModel()

  early_stopping = EarlyStopping(monitor='accuracy', patience= 30)
  model_check = ModelCheckpoint(filepath = OutputDir + "/16_17_lstm_model" + str(test_index+1) +".h5", monitor = 'accuracy', save_best_only=True)
  reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=20)

  
  history = model.fit(train_X, train_y, batch_size = 64, epochs = 50, validation_data = (valid_X, valid_y), callbacks = [model_check, early_stopping,reduct_L_rate]);

  trainning_result.append(calculateScore(train_X, train_y, model));
  validation_result.append(calculateScore(valid_X, valid_y, model));
  testing_result.append(calculateScore(test_X, test_y, model));

  bestModel = load_model(OutputDir + "/16_17_lstm_model" + str(test_index+1) +".h5")
  # score = bestModel.evaluate(test_X, test_y, verbose=0)
  # print("Accuracy: %.2f%%" % (score[1]*100))

  result = bestModel.predict(test_X)
  result = np.array(result)

  # count =0
  idx =0

  for i in range(len(test_set_idxs)):
    # print("id", test_set_idxs[i])
    if test_set_idxs[i] == idx:
      temp_x.append(result[i])
      temp_y =test_y[i]
    else:
      if len(temp_x) > 0:
        max = np.max(temp_x)
        # print("id", idx)
        final_x.append(max)
        final_y.append(temp_y)
        temp_x= []
        temp_y =0
        idx +=1

  if len(temp_x)>0 :
    max = np.max(temp_x)
    final_x.append(max)
    final_y.append(temp_y)
    
  final_x= np.array(final_x)
  final_y= np.array(final_y)

  print("length of final ", len(final_x))
  print("length of final y ", len(final_y))

  tempLabel = np.zeros(shape=final_y.shape, dtype= np.int32)

  for i in range(len(final_y)):
    if final_x[i] <0.5:
      tempLabel[i] = 0
    else:
      tempLabel[i] = 1

  print(tempLabel.shape)

  confusion = confusion_matrix(final_y, tempLabel, labels=[0, 1])
  print("accuracy", (TN+TP)/(TN+FP+FN+TP) )TN, FP, FN, TP = confusion.ravel()
